In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import keras 
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation,Conv2D,MaxPool2D,Flatten,BatchNormalization,MaxPooling2D
from sklearn.preprocessing import LabelEncoder
from skimage.io import imread, imshow
from skimage.transform import resize
from glob import glob
from tensorflow.python.keras.callbacks import ModelCheckpoint

In [2]:
file_path=glob("IDC_regular_ps50_idx5/*/*")
class0=[]
class1=[]
for i in file_path:

    h=glob(i+"/"+"*")
    if i.endswith("0"):
        for j in h:
            class0.append(j)
    else:
        for j in h:
            class1.append(j)

    


In [3]:
sample1=random.sample(class0,18786)
sample2=random.sample(class1,18786)

In [4]:
def feature_extract(path,class_name):
    val=[]
    for i in path:
        image_data=imread(i)
        image_size=resize(image_data,(50,50))
        val.append([image_size,class_name])
    
    return val



In [5]:
non_breast_cancer=feature_extract(sample1,"non_breast_cancer")

In [6]:
breast_cancer=feature_extract(sample2,"breast_cancer")

In [7]:
combined_data=np.concatenate((non_breast_cancer,breast_cancer))
random.seed(42)
random.shuffle(combined_data)


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [8]:
x=[]
y=[]
for features,class_value in combined_data:
    x.append(features)
    y.append(class_value)

In [9]:
x=np.array(x)


In [10]:
lb=LabelEncoder()
y=to_categorical(lb.fit_transform(y))
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [12]:
x_train.shape
K.clear_session()
y.shape

(37572, 2)

In [15]:
model = Sequential()
num_labels=y.shape[1]
model.add(Flatten(input_shape =(50,50,3)))
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.3))
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation("softmax"))

model.compile(optimizer = "Adam",
loss = "binary_crossentropy",
metrics = ["accuracy"])

In [16]:
num_epochs =10
num_batch_size =22

checkpointer = ModelCheckpoint(filepath='breast_cancer_dnn_classification.hdf5',
    verbose=1,
    save_best_only=True)


history= model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)



2022-08-14 15:56:41.207336: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 789000000 exceeds 10% of free system memory.


Epoch 1/10
816/822 [============================>.] - ETA: 0s - loss: 0.4863 - accuracy: 0.8395

2022-08-14 15:56:54.182823: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 338160000 exceeds 10% of free system memory.



Epoch 00001: val_loss improved from inf to 0.43537, saving model to breast_cancer_dnn_classification.hdf5
822/822 [==============================] - 9s 10ms/step - loss: 0.4868 - accuracy: 0.8390 - val_loss: 0.4354 - val_accuracy: 0.8427
Epoch 2/10
819/822 [============================>.] - ETA: 0s - loss: 0.4386 - accuracy: 0.8471
Epoch 00002: val_loss improved from 0.43537 to 0.43515, saving model to breast_cancer_dnn_classification.hdf5
822/822 [==============================] - 6s 8ms/step - loss: 0.4386 - accuracy: 0.8472 - val_loss: 0.4352 - val_accuracy: 0.8427
Epoch 3/10
821/822 [============================>.] - ETA: 0s - loss: 0.4356 - accuracy: 0.8471
Epoch 00003: val_loss did not improve from 0.43515
822/822 [==============================] - 6s 8ms/step - loss: 0.4354 - accuracy: 0.8472 - val_loss: 0.4352 - val_accuracy: 0.8427
Epoch 4/10
815/822 [============================>.] - ETA: 0s - loss: 0.4338 - accuracy: 0.8470
Epoch 00004: val_loss did not improve from 0.43515